In [1]:
from datasets import load_dataset

ds = load_dataset("toughdata/quora-question-answer-dataset")

README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

Quora-QuAD.jsonl:   0%|          | 0.00/60.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

In [4]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 56402
    })
})


In [5]:
#!pip install better-profanity
from better_profanity import profanity
import re
def clean_data(entry):
    answer_length = 1000 <= len(entry['answer']) <= 3000
    word_condition = not re.search(r'\b(I|me|my)\b', entry['answer'], re.IGNORECASE)
    is_clean = (not (profanity.contains_profanity(entry['question']) or profanity.contains_profanity(entry['answer']))) if (answer_length and word_condition) else False
    return answer_length and word_condition and is_clean

clean_ds = ds['train'].filter(clean_data)
print(len(clean_ds))

for i in range(5):
    example = clean_ds[i]
    print("Question:", example['question'])
    print("Answer:", example['answer'])
    print("Answer Length:", len(example['answer']), "characters")
    print("-----\n")

import json
filtered_data = [entry for entry in clean_ds]

# Save the filtered data to a JSON file
with open('quora_clean.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

Filter:   0%|          | 0/56402 [00:00<?, ? examples/s]

4266
Question: What is a proxy, and how can I use one?
Answer: A proxy server is a system or router that provides a gateway between users and the internet. Therefore, it helps prevent cyber attackers from entering a private network. It is a server, referred to as an “intermediary” because it goes between end-users and the web pages they visit online.
 When a computer connects to the internet, it uses an IP address. This is similar to your home’s street address, telling incoming data where to go and marking outgoing data with a return address for other devices to authenticate. A proxy server is essentially a computer on the internet that has an IP address of its own.
 How a Proxy Works
Because a proxy server has its own IP address, it acts as a go-between for a computer and the internet. Your computer knows this address, and when you send a request on the internet, it is routed to the proxy, which then gets the response from the web server and forwards the data from the page to your com

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

7705074

In [15]:
with open('quora_clean.json', 'r', encoding='utf-8') as f:
    quora = json.load(f)

quora[:5]

[{'question': 'What is a proxy, and how can I use one?',
  'answer': 'A proxy server is a system or router that provides a gateway between users and the internet. Therefore, it helps prevent cyber attackers from entering a private network. It is a server, referred to as an “intermediary” because it goes between end-users and the web pages they visit online.\n When a computer connects to the internet, it uses an IP address. This is similar to your home’s street address, telling incoming data where to go and marking outgoing data with a return address for other devices to authenticate. A proxy server is essentially a computer on the internet that has an IP address of its own.\n How a Proxy Works\nBecause a proxy server has its own IP address, it acts as a go-between for a computer and the internet. Your computer knows this address, and when you send a request on the internet, it is routed to the proxy, which then gets the response from the web server and forwards the data from the page t

In [18]:
df = pd.read_json('quora_clean.json')
duplicates = df[df.duplicated('question', keep=False)]
print(len(duplicates))
print(len(df))

3628
4266


In [22]:
import json
from collections import defaultdict

# Load the JSON file
file_path = "quora_clean.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Create a dictionary to store questions and their occurrences
question_dict = defaultdict(list)

# Populate the dictionary
for index, item in enumerate(data):
    question = item['question']
    question_dict[question].append(index)

# Find duplicates
duplicates = {q: indices for q, indices in question_dict.items() if len(indices) > 1}

# Print results
print(f"Total number of questions: {len(data)}")
print(f"Number of unique questions: {len(question_dict)}")
print(f"Number of questions with duplicates: {len(duplicates)}")

# Print some examples of duplicates
print("\nExamples of duplicate questions:")
for i, (question, indices) in enumerate(list(duplicates.items())[:5], 1):
    print(f"\n{i}. Question: {question}")
    print(f"   Occurrences: {len(indices)}")
    print(f"   Indices: {indices}")

# Optional: Save duplicates to a file
duplicates_file = "duplicate_questions.json"
with open(duplicates_file, 'w', encoding='utf-8') as f:
    json.dump(duplicates, f, ensure_ascii=False, indent=2)

print(f"\nFull list of duplicates saved to: {duplicates_file}")

Total number of questions: 4266
Number of unique questions: 1177
Number of questions with duplicates: 539

Examples of duplicate questions:

1. Question: What is a proxy, and how can I use one?
   Occurrences: 29
   Indices: [0, 22, 420, 493, 628, 805, 926, 1646, 1657, 1776, 1844, 2053, 2077, 2086, 2359, 2425, 2803, 2914, 3296, 3381, 3642, 3767, 3933, 4042, 4052, 4093, 4113, 4124, 4239]

2. Question: What are the differences between those who succeed slowly and those who succeed fast?
   Occurrences: 3
   Indices: [2, 1815, 3603]

3. Question: How do I get someone's name by their mobile number?
   Occurrences: 13
   Indices: [3, 109, 271, 311, 363, 1119, 1259, 1818, 2036, 2301, 2480, 2536, 3333]

4. Question: What is an online auction?
   Occurrences: 7
   Indices: [4, 184, 1382, 2000, 3371, 3560, 4102]

5. Question: Why are scientists more concerned about the impact of CO2 in the atmosphere than water vapor?
   Occurrences: 4
   Indices: [6, 720, 1712, 3771]

Full list of duplicates s

In [29]:
import json
from collections import OrderedDict

# Load the JSON file
input_file = "quora_clean.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Original dataset size: {len(data)}")

# Use OrderedDict to remove duplicates while preserving order
seen_questions = set()
unique_data = []

for item in data:
    question = item['question']
    if question not in seen_questions:
        seen_questions.add(question)
        unique_data.append(item)

print(f"Dataset size after removing duplicates: {len(unique_data)}")

# Save the deduplicated data to a new JSON file
output_file = "quora_clean_nodupes.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(unique_data, f, ensure_ascii=False, indent=2)

print(f"Deduplicated dataset saved to: {output_file}")

# Print some statistics
duplicates_removed = len(data) - len(unique_data)
print(f"Number of duplicates removed: {duplicates_removed}")
print(f"Percentage of duplicates: {duplicates_removed / len(data) * 100:.2f}%")

# Optionally, print the first few items of the deduplicated dataset
print("\nFirst 5 items of the deduplicated dataset:")
for i, item in enumerate(unique_data[:5], 1):
    print(f"\n{i}:")
    print(f"Question: {item['question']}")
    print(f"Answer length: {len(item['answer'])}")
    print(f"Answer: {item['answer'][:100]}...")  # Show first 100 characters

Original dataset size: 4266
Dataset size after removing duplicates: 1177
Deduplicated dataset saved to: quora_clean_nodupes.json
Number of duplicates removed: 3089
Percentage of duplicates: 72.41%

First 5 items of the deduplicated dataset:

1:
Question: What is a proxy, and how can I use one?
Answer length: 1678
Answer: A proxy server is a system or router that provides a gateway between users and the internet. Therefo...

2:
Question: Does ethical obligation go beyond standard ethical conduct?
Answer length: 1068
Answer: First of all, an obligation is another word for a contract. If you have a contract you must do it be...

3:
Question: What are the differences between those who succeed slowly and those who succeed fast?
Answer length: 1344
Answer: The difference between those who succeed slowly and those who succeed fast can be attributed to seve...

4:
Question: How do I get someone's name by their mobile number?
Answer length: 1400
Answer: There are several ways to trace someone’s

In [34]:
from initialize import *
from enhanced_hooking import get_activations, add_activations_and_generate, clear_hooks, get_activations_and_generate, zeroout_projections_and_generate
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from enum import Enum
class SteeringType(Enum):
    IN_PROMPT = "In prompt"
    CONTINUOUS = "Continuous"
class AggType(Enum):
    MEANDIFF = "MeanDiff"
    PCA = "PCA"

%load_ext autoreload
%autoreload 2

In [35]:
gc.collect()
torch.cuda.empty_cache()
base_model_path: str = "meta-llama/Meta-Llama-3-8B-Instruct"#"meta-llama/Llama-2-13b-chat-hf"
model_path=base_model_path
###model_path="cackerman/llama2_13b_chat_projection_tune_neg_in"

#device: str = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu" 
#model = load_model(model_path, base_model_path, device)

from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
_ = torch.set_grad_enabled(False)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, token=HF_TOKEN, quantization_config=bnb_config, device_map="auto")
device = model.device
tokenizer = AutoTokenizer.from_pretrained(base_model_path, token=HF_TOKEN)
model.tokenizer = tokenizer
if model.tokenizer.pad_token is None:
    new_pad_token = model.tokenizer.eos_token
    num_added_tokens = model.tokenizer.add_special_tokens({'pad_token': new_pad_token})
    model.resize_token_embeddings(len(model.tokenizer))
    model.config.pad_token_id = model.tokenizer.pad_token_id
model_numlayers = model.config.num_hidden_layers

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def do_batch_decode(generated_tokens, input_ids, tokenizer):
    batch_size = generated_tokens.shape[0]
    start_indices = input_ids.shape[1]
    max_len = generated_tokens.shape[1] - start_indices
    tokens_to_decode = torch.full((batch_size, max_len), tokenizer.pad_token_id, dtype=torch.long)
    
    for i in range(batch_size):
        len_to_decode = generated_tokens.shape[1] - input_ids.shape[1]
        tokens_to_decode[i, :len_to_decode] = generated_tokens[i, input_ids.shape[1]:]
    
    return tokenizer.batch_decode(tokens_to_decode, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
gc.collect()
torch.cuda.empty_cache()
def generate_answers(model, model_name, prompts, batch_size=4, dir_name = "quora_qa"):
    outputfile = f"{model_name}_responses"
    filename = f"{dir_name}/{outputfile}.json"
    partial_filename = f"{dir_name}/{outputfile}_partial.json"    

    results = []    
    if os.path.exists(filename) or os.path.exists(partial_filename):
        results_tmp = load_from_json(filename) if os.path.exists(filename) else load_from_json(partial_filename)
        results = [result for result in results_tmp if result['answer'] != ""]
        existing_qs = {result['question'] for result in results}
        prompts = [q for q in prompts if q['question'] not in existing_qs]
        if prompts == []:
            print(f"All data in {filename} has been processed. Exiting.")
            return
        else:
            print(f"Processing new data not in {filename}")

    model.eval()
    model.tokenizer.padding_side="left"
    sampling_kwargs={"use_cache": True, "pad_token_id": model.tokenizer.pad_token_id, "max_new_tokens": 1000, "do_sample": True}
    initlenresults=0
    
    for i in tqdm(range(initlenresults, initlenresults+len(prompts), batch_size)):
        batch = prompts[i-initlenresults:min(i-initlenresults+batch_size,len(prompts))]
        input_texts = [LLAMA3_PROMPT_TEMPLATE.format(system_prompt="",user_prompt=item["question"]+"\n") for item in batch] 
        inputs = model.tokenizer(input_texts, return_tensors="pt", truncation=True, padding=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        with torch.no_grad():
            output_ids = model.generate(**inputs, **sampling_kwargs)

        output = do_batch_decode(output_ids, inputs['input_ids'], model.tokenizer)

        for j, item in enumerate(batch):
            if output[j].strip() == "": 
                print("Bad output at ",item["question"])
                continue
            if output[j].strip()[:2] == "I ":
                print(f"Refusal at {item['question']}: {output[j]}")
                continue
            results.append({"id": f"id{i+j}", "question": item["question"], "answer": output[j].strip()})
            
        if (i-initlenresults + batch_size) % 8 == 0 or (i-initlenresults + batch_size) >= len(prompts):
            print(f"Completed {i-initlenresults + batch_size} rows out of {len(prompts)}")
            save_to_json(results, partial_filename)

    save_to_json(results, filename)

input_file = "quora_clean_nodupes.json"
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)
generate_answers(model, "llama3_8bchat", data, batch_size=8)


Processing new data not in quora_qa/llama3_8bchat_responses.json


  1%|          | 1/146 [01:44<4:11:47, 104.19s/it]

Refusal at How do I get someone's name by their mobile number?: 

I cannot provide information that would assist in obtaining someone's personal information, such as their name, without their consent.
Refusal at How would I start a boiler room stock scheme?: 

I cannot provide information or guidance on illegal or harmful activities, including starting a boiler room stock scheme. Can I help you with something else?
Completed 8 rows out of 1163


  1%|▏         | 2/146 [03:08<3:42:28, 92.70s/it] 

Completed 16 rows out of 1163


...

Completed 80 rows out of 1163


  8%|▊         | 11/146 [18:25<3:46:45, 100.78s/it]

Completed 88 rows out of 1163


  8%|▊         | 12/146 [20:06<3:44:46, 100.65s/it]

Completed 96 rows out of 1163
